In [1]:
import minsearch
import json
from openai import OpenAI

client = OpenAI(api_key='x')

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)


index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)


def search(query):

    boost = {'question': 3.0, 'section': 0.5} # Peso das seções
    filter = {'course': 'data-engineering-zoomcamp'}

    results = index.search(
        query=query,
        boost_dict=boost,
        filter_dict=filter,
        num_results=5
    )

    return results


def build_prompt(query, search_results):


    prompt_template = """

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.

Use only the facts from the CONTEXT when answering the QUESTION.

If the CONTEXT doesn't contain the answer, output NONE.


QUESTION: {question}

CONTEXT:
{context}



"""

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context)

    return prompt


def llm_call(prompt):

    response = client.chat.completions.create(
                model='gpt-3.5-turbo', # 'gpt-4o' pago
                messages=[{"role": "user", "content": prompt}]
            ) # Resposta genérica (Erro porque não coloquei o cartão ainda pra billar)
    
    content = response.choices[0].content
    
    return content

def rag_flow(query):
    search_results = search(query=query)
    prompt = build_prompt(query=query, search_results=search_results)
    answer = llm_call(prompt=prompt)


### Teste das funções 

In [2]:
search('How do I run Kafka?')

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi

In [3]:
search_results = search('How do I run Kafka?')


print(build_prompt(query='How do I run Kafka?', search_results=search_results))



You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.

Use only the facts from the CONTEXT when answering the QUESTION.

If the CONTEXT doesn't contain the answer, output NONE.


QUESTION: How do I run Kafka?

CONTEXT:
section: Module 6: streaming with kafka
question: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
answer: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

section: Module 6: streaming with kafka
question: Module “kafka” not found when trying to run producer.py
answer: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.
To create a virtual env and install packages (run only once)
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
To activate it (you'll need to run it every time you need the virtual env):
source env/bin/activate
To deactivate it:
deactivate
Thi

### Fluxo RAG

In [4]:
query = 'The course already started. Can I still enroll?' # 'How do I run Kafka?'
search_results = search(query=query)
prompt = build_prompt(query=query, search_results=search_results)
answer = llm_call(prompt=prompt)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: x. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}